In [ ]:
#!pip install lxml
#!pip install requests
#!pip install beautifulsoup4

In [222]:
import requests
import os
from collections import defaultdict
from bs4 import BeautifulSoup
#
from read_parameters_2 import read_excel_parameters

In [223]:
def convert_month_to_number(value_name):
    Months=['enero','febrero','marzo','abril','mayo','junio','julio','agosto','setiembre','septiembre','octubre','noviembre','diciembre']
    Months_map=['01','02','03','04','05','06','07','08','09','09','10','11','12']

    months_dict=dict(zip(Months, Months_map))

    names_dict_new=[]

    for month in Months:
        if month in value_name.lower():
            names_dict_new.append(months_dict.get(month,"There's no value for that month in 'months_dict'"))    
    
    num_dict_new = ""
    for i in names_dict_new:
        num_dict_new=num_dict_new+i+"_"
        
    return num_dict_new[:-1]

In [224]:
from openpyxl import load_workbook
from io import BytesIO
import urllib

def load_workbook_from_url(url):
    response_content = urllib.request.urlopen(url).read()
    return response_content, load_workbook(filename = BytesIO(response_content))

In [324]:
def make_df(path,sheet,category,dep,year,max_column):
    
    [skiprows,names,nrows,skipfooter,usecols]=read_excel_parameters(year=year,
                                                            category=category,
                                                            dep=dep,
                                                            max_column=max_column)
    
    
    df=pd.read_excel(path,
                     sheet_name=dep,
                     skiprows=skiprows,
                     usecols=usecols,
                     names=names,
                     nrows=nrows,
                     skipfooter=skipfooter,
                     engine='openpyxl',
                     header=None)

    df.replace(['*','(SD)'], 0 , inplace=True)
    df.fillna(method="ffill",inplace=True)
    
    return df

In [309]:
def save_excel_file(response_content,category,year,month_ranges):
    excel_file_name=f"{category}_{year}_{month_ranges}"
    path=f"./data/{year}/{excel_file_name}.xlsx"
    open(path, "wb").write(response_content)
    
    print(f"\t{path} file saved")
    
    return path

In [340]:
def main(URL,years):
    """
    URL: website URL 
    years: is a list of ranges of years
    """
    ### WebScraping
    html_text=requests.get(URL).text
    soup=BeautifulSoup(html_text,'lxml')
    block=soup.find("main",class_="l-bl block block-main-content") 
    li_tags= block.find_all('li')

    first_filter = "Indicadores"

    year_list=years
    second_filter=[str(i)for i in year_list]

    names_dict=defaultdict(list)
    urls_dict=defaultdict(list)

    for li_tag in li_tags:
        li_text = li_tag.a.text.replace("\xa0", " ")
        if first_filter in li_text:
            if any((match := substring) in li_text for substring in second_filter):
                names_dict[match].append(li_text)
                li_url = li_tag.a['href']
                urls_dict[match].append(li_url)         
    ###
            
    if not os.path.exists("./data"):
        os.mkdir("./data")  

    years = names_dict.keys() #urls_dict.keys()
    
    for year in years:
        print(f"{year}:")
        if not os.path.exists(f"./data/{year}"):
            os.mkdir(f"./data/{year}")  

        value_name_list=names_dict.get(year,"there's no value_name for that year")
        value_url_list=urls_dict.get(year,"there's no value_url for that year")

        for i,value_name in enumerate(value_name_list):
            
            month_ranges = convert_month_to_number(value_name)

            if "Gestantes" in value_name:
                category="pregnant_women"
                
                if "web.ins.gob.pe" in value_url_list[i]:
                    print(f"\t{category}_if")
                    URL=value_url_list[i]
                    print(f"\tURL: {URL}")
                    
                    response_content , wb = load_workbook_from_url(URL)
                    path = save_excel_file(response_content,category,year,month_ranges)                    
                    sheets= wb.sheetnames[1:]

                    for sheet in sheets:
                        dep=sheet
                        max_column =wb[sheet].max_column
                        df = make_df(path,sheet,category,dep,year,max_column)
                        #display(df)
                        df_file_name=f"{category}_{dep}_{year}_{month_ranges}"    
                        print(f"\t\t{df_file_name}")
                    print()
                    continue
                else:
                    print(f"\t{category}_else")
                    URL="https://web.ins.gob.pe"+value_url_list[i]
                    print(f"\tURL: {URL}")
                    
                    response_content , wb =load_workbook_from_url(URL)
                    path = save_excel_file(response_content,category,year,month_ranges)                    
                    sheets= wb.sheetnames[1:]

                    for sheet in sheets:
                        dep=sheet
                        max_column =wb[sheet].max_column
                        #print(dep,max_column)
                        df = make_df(path,sheet,category,dep,year,max_column)                        
                        #display(df)
                        df_file_name=f"{category}_{dep}_{year}_{month_ranges}"    
                        print(f"\t\t{df_file_name}")
                    print()
                    continue

            elif ("Niños Venezolanos" in value_name) or ("Niños Extranjeros" in value_name):
                category="foreign_children"

                if "web.ins.gob.pe" in value_url_list[i]:
                    print(f"\t{category}_if")
                    URL=value_url_list[i]
                    print(f"\tURL: {URL}")
                    
                    response_content , wb =load_workbook_from_url(URL)
                    path = save_excel_file(response_content,category,year,month_ranges)                    
                    sheets= wb.sheetnames[1:]

                    for sheet in sheets:
                        dep=sheet
                        max_column =wb[sheet].max_column
                        df = make_df(path,sheet,category,dep,year,max_column)
                        #display(df)
                        df_file_name=f"{category}_{dep}_{year}_{month_ranges}"    
                        print(f"\t\t{df_file_name}")
                    print()
                    continue
                else:
                    print(f"\t{category}_else")
                    URL="https://web.ins.gob.pe"+value_url_list[i]
                    print(f"\tURL: {URL}")
                    
                    response_content , wb =load_workbook_from_url(URL)
                    path = save_excel_file(response_content,category,year,month_ranges)                    
                    sheets= wb.sheetnames[1:]

                    for sheet in sheets:
                        dep=sheet
                        max_column =wb[sheet].max_column
                        df = make_df(path,sheet,category,dep,year,max_column)
                        #display(df)
                        df_file_name=f"{category}_{dep}_{year}_{month_ranges}"    
                        print(f"\t\t{df_file_name}")
                    print()
                    continue
                    
            elif "Niños VRAEM" in value_name:
                category="VRAEM_children"

                if "web.ins.gob.pe" in value_url_list[i]:
                    print(f"\t{category}_if")
                    URL=value_url_list[i]
                    print(f"\tURL: {URL}")
                    
                    response_content , wb =load_workbook_from_url(URL)
                    path = save_excel_file(response_content,category,year,month_ranges)                    
                    sheets= wb.sheetnames[1:]

                    for sheet in sheets:
                        dep=sheet
                        max_column =wb[sheet].max_column
                        df = make_df(path,sheet,category,dep,year,max_column)
                        #display(df)
                        df_file_name=f"{category}_{dep}_{year}_{month_ranges}"    
                        print(f"\t\t{df_file_name}")
                    print()
                    continue
                else:
                    print(f"\t{category}_else")
                    URL="https://web.ins.gob.pe"+value_url_list[i]
                    print(f"\tURL: {URL}")
                    
                    response_content , wb =load_workbook_from_url(URL)
                    path = save_excel_file(response_content,category,year,month_ranges)                    
                    sheets= wb.sheetnames[1:]

                    for sheet in sheets:
                        dep=sheet
                        max_column =wb[sheet].max_column
                        df = make_df(path,sheet,category,dep,year,max_column)
                        #display(df)
                        df_file_name=f"{category}_{dep}_{year}_{month_ranges}"    
                        print(f"\t\t{df_file_name}")
                    print()
                    continue
            else:
                category="children"
                
                if "web.ins.gob.pe" in value_url_list[i]:
                    print(f"\t{category}_if")
                    URL=value_url_list[i]
                    print(f"\tURL: {URL}")
                    
                    response_content , wb =load_workbook_from_url(URL)
                    path = save_excel_file(response_content,category,year,month_ranges)                    
                    sheets= wb.sheetnames[1:]

                    for sheet in sheets:
                        dep=sheet
                        max_column =wb[sheet].max_column
                        df = make_df(path,sheet,category,dep,year,max_column)
                        #display(df)
                        df_file_name=f"{category}_{dep}_{year}_{month_ranges}"    
                        print(f"\t\t{df_file_name}")
                    print()
                    continue
                elif "gob.pe" in value_url_list[i]:
                    print(f"\t{category}_elif")
                    URL="https://cdn.www.gob.pe/uploads/document/file/3566498/1.Indic%20Ni%C3%B1os%20a%20Junio%202022%20-%20PERU.xlsx?v=1661961860"
                    print(f"\tURL: {URL}")
                    
                    response_content , wb =load_workbook_from_url(URL)
                    path = save_excel_file(response_content,category,year,month_ranges)                    
                    sheets= wb.sheetnames[1:]

                    for sheet in sheets:
                        dep=sheet
                        max_column =wb[sheet].max_column
                        df = make_df(path,sheet,category,dep,year,max_column)
                        #display(df)
                        df_file_name=f"{category}_{dep}_{year}_{month_ranges}"    
                        print(f"\t\t{df_file_name}")
                    print()
                    continue
                else:
                    print(f"\t{category}_else")
                    URL="https://web.ins.gob.pe"+value_url_list[i]
                    print(f"\tURL: {URL}")
                    
                    response_content , wb =load_workbook_from_url(URL)
                    path = save_excel_file(response_content,category,year,month_ranges)                    
                    sheets= wb.sheetnames[1:]

                    for sheet in sheets:
                        dep=sheet
                        max_column =wb[sheet].max_column
                        df = make_df(path,sheet,category,dep,year,max_column)
                        #display(df)
                        df_file_name=f"{category}_{dep}_{year}_{month_ranges}"    
                        print(f"\t\t{df_file_name}")
                    print()
                    continue

        print()
    print("End of program")

In [341]:
years=list(range(2021,2023))
URL="https://web.ins.gob.pe/es/alimentacion-y-nutricion/vigilancia-alimentaria-y-nutricional/vigilancia-del-sistema-de-informacion-del-estado-nutricional-en-%20EESS"
main(URL,years=years)


2022:
	pregnant_women_else
	URL: https://web.ins.gob.pe/sites/default/files/Archivos/cenan/van/informes/2022/2.Indic%20Gestantes%20Enero_Septiembre%202022.xlsx
	./data/2022/pregnant_women_2022_01_09.xlsx file saved
		pregnant_women_IMC_PG-CLAP x DEP_2022_01_09
		pregnant_women_IMC_PG-CLAP x DIRESA_2022_01_09
		pregnant_women_IMC_PG-CLAP x DISTRITO_2022_01_09
		pregnant_women_Anemia x DEP_2022_01_09
		pregnant_women_Anemia x DIRESA_2022_01_09
		pregnant_women_Anemia x DISTRITO_2022_01_09

	children_else
	URL: https://web.ins.gob.pe/sites/default/files/Archivos/cenan/van/informes/2022/1.Indic%20Ni%C3%B1os%20a%20Septiembre%202022%20-%20PERU.xlsx
	./data/2022/children_2022_01_09.xlsx file saved
		children_EN 0-35m x DEP_2022_01_09
		children_EN 0-35m x DIRESA_2022_01_09
		children_EN 0-35m x DISTRITO_2022_01_09
		children_EN 0-59m x DEP_2022_01_09
		children_EN 0-59m x DIRESA_2022_01_09
		children_EN 0-59m x DISTRITO_2022_01_09
		children_Anemia 6-35m x DEP_2022_01_09
		children_Anemia 6-3

		foreign_children_EN 0-35m x DIRESA_2021_01_12
		foreign_children_EN 0-35m x DISTRITO_2021_01_12
		foreign_children_EN 0-59m x DEP_2021_01_12
		foreign_children_EN 0-59m x DIRESA_2021_01_12
		foreign_children_EN 0-59m x DISTRITO_2021_01_12
		foreign_children_Anemia 6-35m x DEP_2021_01_12
		foreign_children_Anemia 6-35m x DIRESA_2021_01_12
		foreign_children_Anemia 6-35m x DISTRITO_2021_01_12
		foreign_children_Anemia 6-59m x DEP_2021_01_12
		foreign_children_Anemia 6-59m x DIRESA_2021_01_12
		foreign_children_Anemia 6-59m x DISTRITO_2021_01_12

	VRAEM_children_else
	URL: https://web.ins.gob.pe/sites/default/files/Archivos/cenan/van/informes/2021/1.Indic%20Ni%C3%B1os%20a%20Diciembre%202021%20-%20VRAEM_0.xlsx
	./data/2021/VRAEM_children_2021_01_12.xlsx file saved
		VRAEM_children_EN 0-35m x DISTRITO_2021_01_12
		VRAEM_children_EN 0-59m x DISTRITO_2021_01_12
		VRAEM_children_Anemia 6-35m x DISTRITO_2021_01_12
		VRAEM_children_Anemia 6-59m x DISTRITO_2021_01_12

	children_else
	URL: https

		children_EN 0-35m x DIRESA_2021_01_07
		children_EN 0-35m x DISTRITO_2021_01_07
		children_EN 0-59m x DEP_2021_01_07
		children_EN 0-59m x DIRESA_2021_01_07
		children_EN 0-59m x DISTRITO_2021_01_07
		children_Anemia 6-35m x DEP_2021_01_07
		children_Anemia 6-35m x DIRESA_2021_01_07
		children_Anemia 6-35m x DISTRITO_2021_01_07
		children_Anemia 6-59m x DEP_2021_01_07
		children_Anemia 6-59m x DIRESA_2021_01_07
		children_Anemia 6-59m x DISTRITO_2021_01_07

	pregnant_women_else
	URL: https://web.ins.gob.pe/sites/default/files/Archivos/cenan/van/informes/2021/1.Indic%20Gestantes%20Enero_Junio%202021.xlsx
	./data/2021/pregnant_women_2021_01_06.xlsx file saved
		pregnant_women_IMC_PG-CLAP x DEP_2021_01_06
		pregnant_women_IMC_PG-CLAP x DIRESA_2021_01_06
		pregnant_women_IMC_PG-CLAP x DISTRITO_2021_01_06
		pregnant_women_Anemia x DEP_2021_01_06
		pregnant_women_Anemia x DIRESA_2021_01_06
		pregnant_women_Anemia x DISTRITO_2021_01_06

	children_else
	URL: https://web.ins.gob.pe/sites/defa